In [1]:
'''Go through movie->features pipeline manually'''
# First load the stimulus
from featurex.tests.utils import get_test_data_path
from os.path import join
from featurex.stimuli.video import VideoStim
from featurex.converters.video import VideoToAudioConverter, FrameSamplingConverter
from featurex.converters.image import TesseractConverter
from featurex.converters.api import WitTranscriptionConverter
from featurex.extractors import ExtractorResult
from featurex.extractors.image import VibranceExtractor
from featurex.extractors.text import LengthExtractor
from featurex.graph import Graph, Node

filename = join(get_test_data_path(), 'video', 'obama_speech.mp4')
video = VideoStim(filename)

In [2]:
# Extract the audio
conv = VideoToAudioConverter()
audio = conv.convert(video)

[MoviePy] Writing audio in /Users/quinnmac/Library/Python/2.7/lib/python/site-packages/featurex-0.0.1-py2.7.egg/featurex/tests/data/video/obama_speech.wav


100%|██████████| 193/193 [00:00<00:00, 1478.78it/s]

[MoviePy] Done.


In [3]:
# Subsample the video
conv = FrameSamplingConverter(every=15)
derived = conv.convert(video)

In [4]:
# Extract text from the video
conv = TesseractConverter()
visual_texts = []
for frame in derived:
    visual_texts.append(conv.convert(frame))

In [5]:
# Extract text from the audio
conv = WitTranscriptionConverter()
audio_text = conv.convert(audio)

In [6]:
# Extract a low-level image feature from each frame
ext = VibranceExtractor()
visual_features = []
for frame in derived:
    visual_features.append(ext.extract(frame))

In [8]:
# Extract word length from both audio and visual text
ext = LengthExtractor()
visual_length = [ext.extract(t) for t in visual_texts]
audio_length = [ext.extract(t) for t in audio_text]

In [7]:
# Merge and display results
print ExtractorResult.merge_stims(visual_features)
print ExtractorResult.merge_stims(visual_length)
print ExtractorResult.merge_stims(audio_length)

        onset  duration    vibrance
  stim                             
0 0       NaN       NaN  418.851528
  15      NaN       NaN  441.618626
  30      NaN       NaN  506.321013
  45      NaN       NaN  575.903070
  60      NaN       NaN  586.500651
  75      NaN       NaN  590.007483
  90      NaN       NaN  589.524823
                                                      onset  duration  \
  stim                                                                  
0                                                       NaN       NaN   
                                                        NaN       NaN   
                                                        NaN       NaN   
  mslnsu-r onu‘ﬁ. SAVEIENV nu IRAN                      NaN       NaN   
  1'"\nPIESIIIE Y' I' I ‘ s\nt 1'. I)“; 555'} HEM...    NaN       NaN   
                                                        NaN       NaN   
  , 7 V\nPnEslnEMJ'W min: an Inn:\nA ‘                  NaN       NaN   

                  

In [2]:
# Graph way of doing the same pipeline
visual_nodes = [(FrameSamplingConverter(every=15), 'framesampling', 
             [(TesseractConverter(), 'visual_text', 
                [(LengthExtractor(), 'text_length')]), 
              (VibranceExtractor(), 'visual_vibrance')])]
audio_nodes = [(VideoToAudioConverter(), 'audio', 
            [(WitTranscriptionConverter(), 'audio_text', 
              [(LengthExtractor(), 'text_length')])])]

graph = Graph()
graph.add_children(visual_nodes)
graph.add_children(audio_nodes)
graph.extract(video)

[MoviePy] Writing audio in /Users/quinnmac/Library/Python/2.7/lib/python/site-packages/featurex-0.0.1-py2.7.egg/featurex/tests/data/video/obama_speech.wav


100%|██████████| 193/193 [00:00<00:00, 1874.24it/s]

[MoviePy] Done.


AttributeError: 'list' object has no attribute 'stim'